In [ ]:
import pvl
import struct
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os.path
import binascii

In [ ]:
kaguya_file = '/Path/to/kaguyami/or/kaguyatc.img'
image_file = kaguya_file

In [ ]:
header = pvl.load(kaguya_file)

In [ ]:
header

In [ ]:
header["^IMAGE"]

In [ ]:
with open(kaguya_file, 'rb') as f:
    try:
        # For marci image 
        # image_offset = header['^IMAGE'] * header['LABEL_RECORDS'] * header["RECORD_BYTES"] - (header['IMAGE']['SAMPLE_BITS']//8)
        
        # For others
        image_offset = header["^IMAGE"].value-(header['IMAGE']['SAMPLE_BITS']//8) 
        
        f.seek(image_offset)
        b_image_data = f.read()
    except AttributeError:
        # If detached label, "^IMAGE" will be a list.
        image_file = os.path.dirname(kaguya_file) + "/" + header["^IMAGE"][0]
        image_offset = header["^IMAGE"][1].value
        with open(image_file, 'rb') as im_f:
            b_image_data = im_f.read()

In [ ]:
n_lines = 20
n_bands = 2
if "BANDS" not in header["IMAGE"].keys():
    n_bands = 1
line_length = header['IMAGE']['LINE_SAMPLES'] * (header['IMAGE']['SAMPLE_BITS']//8)

In [ ]:
image_data = []
for i in range(n_bands):
    image_data.append([])
    for j in range(n_lines):
        image_sample = np.frombuffer(b_image_data[j*line_length:(j+1)*line_length], dtype=np.int16, count=int(line_length/2))
        image_data[i].append(image_sample)
image_data = np.array(image_data)

In [ ]:
image_data.min()

In [ ]:
plt.imshow(image_data[0])

In [ ]:
class RealIsisCubeLabelEncoder(pvl.encoder.ISISEncoder):    
    def encode_time(self, value):
        if value.microsecond:
            second = u'%02d.%06d' % (value.second, value.microsecond)
        else:
            second = u'%02d' % value.second

        time = u'%02d:%02d:%s' % (value.hour, value.minute, second)
        return time

In [ ]:
image_fn, image_ext = os.path.splitext(image_file)
crop = '_cropped'
mini_image_fn = image_fn + crop + image_ext
mini_image_bn = os.path.basename(mini_image_fn)

grammar = pvl.grammar.ISISGrammar()
grammar.comments+=(("#", "\n"), )
encoder = RealIsisCubeLabelEncoder()

# Overwrite the number of lines in the label
header['IMAGE']['LINES'] = n_lines
header['IMAGE']['BANDS'] = n_bands

if kaguya_file != image_file:
    # If detached label, point the mini label to the mini image
    header['^IMAGE'] = [mini_image_bn, pvlcollections.Units(1, 'BYTES')]
    header['FILE_NAME'] = mini_image_bn
else:
    # If attached label, calculate the new offset
    header['^IMAGE'] = pvl.collections.Units(len(pvl.dumps(header, encoder=encoder, grammar=grammar)), 'BYTES')

In [ ]:
label_fn, label_ext = os.path.splitext(kaguya_file)
out_label = label_fn + crop + label_ext
print(out_label)

pvl.dump(header, out_label, encoder=encoder, grammar=grammar)

In [ ]:
with open(mini_image_fn, 'ab+') as f:
    b_reduced_image_data = image_data.tobytes()
    f.seek(0, 2)
    f.write(b_reduced_image_data)

In [ ]:
pvl.load(mini_image_fn)